In [ ]:
import numpy as np
import pandas as pd

In [ ]:
seed = 0
np.random.seed(seed)

## Defines the base dataset and ground truth label

In [ ]:
class SaddlePoint2D():
    def __init__(self, x1_range = 10, x2_range = 10, rescale_y_max = 30, rescale_y_int = 10):            
        
        self.x1_int = np.random.uniform(low=10, high=15, size=1).round(1)
        self.x2_int = np.random.uniform(low=10, high=15, size=1).round(1)
        self.y_int = 30
        self.x1_range = x1_range
        self.x2_range = x2_range
        
        self.x2_int_rescaled = self.x2_int
        
        x2_coef = 1
        if np.random.uniform() > 0.5:
            x2_coef = -1
        x1_coef = -1 * x2_coef
        
        self.x1_coef = x1_coef
        self.x2_coef = x2_coef
        
        self.rescale_y_max = rescale_y_max
        self.rescale_y_int = rescale_y_int
        
        x1_extrema_vals = [x1_int - x1_range / 2, x1_int, x1_int + x1_range / 2]
        x2_extrema_vals = [x2_int - x2_range / 2, x2_int, x2_int + x2_range / 2]
                
    def generate_y(self, X1, X2):
        return self.x1_coef * (X1 - self.x1_int)**2 + self.x2_coef * (X2 - self.x2_int)**2 + self.y_int
    
    def gt_X2_increasing(self, X2):
        '''
        Returns ground truth label for a given point
        '''
        if self.x2_coef == -1:
            return X2 <= self.x2_int
        elif self.x2_coef == 1:
            return X2 > self.x2_int
        
    def rescale_array(self, arr, new_range, new_intercept):
        return new_range * arr - new_range / 2 + new_intercept

    def generate_random_data(self, N, standardize = False):
        '''
        Returns N points sampled from y = (X1 - x1_int)^2 - (X2 - x2_int)^2
        '''
        X1 = self.rescale_array(np.random.rand(N), self.x1_range, self.x1_int)
        X2 = self.rescale_array(np.random.rand(N), self.x2_range, self.x2_int)

        y = self.generate_y(X1, X2)

        X = np.stack((X1, X2), axis=1)

        gt_increasing = self.gt_X2_increasing(X2)

        return X, y, gt_increasing

    def sample_new_point(self):
        shift = (15-10)/4
        X1_explain = [np.random.uniform(low=10 + shift, high=15- shift, size=1)[0]]
        X2_explain = [np.random.uniform(low=10 + shift, high=15- shift, size=1)[0]]
        X_explain = np.stack((X1_explain, X2_explain), axis=1)
        y_explain = self.generate_y(X1_explain, X2_explain)
        u_explain = self.gt_X2_increasing(X2_explain)
        
        return X_explain, y_explain, u_explain

## Define base model

In [ ]:
import lightgbm as lgb

def get_base_model():
    pred_model = lgb.LGBMRegressor()
    return pred_model